In [10]:
import numpy as np
import cv2
from collections import Counter

img_path = 'sar_1_gray.jpg'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

img = img.astype(np.float32)
np.savetxt('txt_image.txt', img, fmt='%d')

# Вейвлет-преобразования Хаара
def haar_transform(image):
    rows = image.shape[0] - image.shape[0] % 2
    cols = image.shape[1] - image.shape[1] % 2
    image = image[:rows, :cols]
    row_transform = np.zeros_like(image, dtype=np.float32)
    for i in range(rows):
        for j in range(0, cols - 1, 2):
            row_transform[i, j // 2] = (image[i, j] + image[i, j + 1]) / 2
            row_transform[i, (j // 2) + cols // 2] = (image[i, j] - image[i, j + 1]) / 2
    result = np.zeros_like(row_transform, dtype=np.float32)
    for j in range(cols):
        for i in range(0, rows - 1, 2):
            result[i // 2, j] = (row_transform[i, j] + row_transform[i + 1, j]) / 2
            result[(i // 2) + rows // 2, j] = (row_transform[i, j] - row_transform[i + 1, j]) / 2
    LL = result[:rows // 2, :cols // 2]  # Низкие частоты
    LH = result[:rows // 2, cols // 2:]  # Горизонтальные высокие частоты
    HL = result[rows // 2:, :cols // 2]  # Вертикальные высокие частоты
    HH = result[rows // 2:, cols // 2:]  # Диагональные высокие частоты

    return LL, LH, HL, HH

LL, LH, HL, HH = haar_transform(img)
n_quants = 5

def quantize(coeffs, n_quants):
    min_val = np.min(coeffs)
    max_val = np.max(coeffs)
    step = (max_val - min_val) / n_quants
    quantized = np.round((coeffs - min_val) / step).astype(int)
    return quantized, min_val, step

LH_q, LH_min, LH_step = quantize(LH, n_quants)
HL_q, HL_min, HL_step = quantize(HL, n_quants)
HH_q, HH_min, HH_step = quantize(HH, n_quants)

def run_length_encode(data):
    encoded = []
    for value, count in Counter(data.flatten()).items():
        encoded.append((value, count))
    return encoded

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

with open('wavelet_image.txt', 'w') as f:
    np.savetxt(f, LL, fmt='%d')
    f.write('\n')
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")

original_size = img.nbytes
compressed_size = 0
with open('wavelet_image.txt', 'r') as f:
    compressed_size = len(f.read().encode('utf-8'))

print(f"Размер исходного изображения: {original_size} байт")
print(f"Размер сжатого изображения: {compressed_size} байт")
print(f"Степень сжатия: {original_size / compressed_size:.2f}")

Размер исходного изображения: 960000 байт
Размер сжатого изображения: 190901 байт
Степень сжатия: 5.03
